# Exploring Orthology data for BioThings Studio Parser

In [95]:
import pandas as pd
import numpy as np
import os, glob, csv, re
import gzip
import shutil

In [38]:
# we'll remove space in keys to make queries easier. Also, lowercase is preferred
# for a BioThings API. We'll an helper function from BioThings SDK
process_key = lambda k: k.replace(" ","_").lower()

---
## Load input data   

Datasource: ```https://fms.alliancegenome.org/download/ORTHOLOGY-ALLIANCE_COMBINED.tsv.gz```     
  
  
*The first lines (0-14) are summary information and are disregarded when formatting the dataframe.*

In [51]:
infile = os.path.join("c:\\Users\\19802\\Documents\\dev\scripps\\BioThings\\ORTHOLOGY-ALLIANCE_COMBINED.tsv.gz")
assert os.path.exists(infile)

In [41]:
infile2 = os.path.join("c:\\Users\\19802\\Documents\\dev\scripps\\BioThings\\drugLabels.byGene.tsv")
assert os.path.exists(infile2)

In [70]:
# open and print the summary key at top of file
with gzip.open(infile, 'rb') as f_in:
    for line in f_in.readlines()[:15]:
        print(line)
    print("\n")

# open and load tsv into dataframe
with gzip.open(infile, 'rb') as f_in:
    data=pd.read_csv(f_in, header=15, sep="\\t")
data_list=data.to_dict(orient='records')

print("[INFO] completed loading data.")


b'##########################################################################\n'
b'#\n'
b'# Data type: Orthology\n'
b'# Data format: tsv\n'
b'# README: \n'
b'# Source: Alliance of Genome Resources (Alliance)\n'
b'# Source URL: http://alliancegenome.org/downloads\n'
b'# Help Desk: help@alliancegenome.org\n'
b'# Orthology Filter: Stringent\n'
b'# Taxon IDs: NCBITaxon:9606, NCBITaxon:10116, NCBITaxon:10090, NCBITaxon:7955, NCBITaxon:7227, NCBITaxon:6239, NCBITaxon:559292\n'
b'# Species: Homo sapiens, Rattus norvegicus, Mus musculus, Danio rerio, Drosophila melanogaster, Caenorhabditis elegans, Saccharomyces cerevisiae\n'
b'# Alliance Database Version: 4.1.0\n'
b'# Date file generated (UTC): 2021-08-10 17:59\n'
b'#\n'
b'##########################################################################\n'


[INFO] completed loading data.


In [42]:
data2=pd.read_csv(infile2, sep="\t",squeeze=True,quoting=csv.QUOTE_NONE).to_dict(orient='records') 


## Explore Data  


In [101]:
# Count unique values found in columns, if under a certain threshold view the unique list of values
for col in data.columns.values:
    list_len=len(data[col].unique().tolist())
    print("column: %s length: %s"%(col, list_len))
    if list_len < 20:
        print("unique values: ", data[col].unique().tolist())
    print("----")


column: Gene1ID length: 100430
----
column: Gene1Symbol length: 81990
----
column: Gene1SpeciesTaxonID length: 7
unique values:  ['NCBITaxon:6239', 'NCBITaxon:7955', 'NCBITaxon:10116', 'NCBITaxon:10090', 'NCBITaxon:9606', 'NCBITaxon:7227', 'NCBITaxon:559292']
----
column: Gene1SpeciesName length: 7
unique values:  ['Caenorhabditis elegans', 'Danio rerio', 'Rattus norvegicus', 'Mus musculus', 'Homo sapiens', 'Drosophila melanogaster', 'Saccharomyces cerevisiae']
----
column: Gene2ID length: 100430
----
column: Gene2Symbol length: 81990
----
column: Gene2SpeciesTaxonID length: 7
unique values:  ['NCBITaxon:559292', 'NCBITaxon:7227', 'NCBITaxon:7955', 'NCBITaxon:10116', 'NCBITaxon:9606', 'NCBITaxon:6239', 'NCBITaxon:10090']
----
column: Gene2SpeciesName length: 7
unique values:  ['Saccharomyces cerevisiae', 'Drosophila melanogaster', 'Danio rerio', 'Rattus norvegicus', 'Homo sapiens', 'Caenorhabditis elegans', 'Mus musculus']
----
column: Algorithms length: 2529
----
column: AlgorithmsMat

In [43]:
data2[0]

{'Gene ID': 'PA267',
 'Gene Symbol': 'ABCB1',
 'Label IDs': 'PA166123409;PA166159586',
 'Label Names': 'Annotation of EMA Label for aliskiren and ABCB1;Annotation of HCSC Label for aliskiren and ABCB1'}

In [25]:
# view first record
data_list[0]

{'Gene1ID': 'WB:WBGene00011502',
 'Gene1Symbol': 'vps-53',
 'Gene1SpeciesTaxonID': 'NCBITaxon:6239',
 'Gene1SpeciesName': 'Caenorhabditis elegans',
 'Gene2ID': 'SGD:S000003566',
 'Gene2Symbol': 'VPS53',
 'Gene2SpeciesTaxonID': 'NCBITaxon:559292',
 'Gene2SpeciesName': 'Saccharomyces cerevisiae',
 'Algorithms': 'PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roundup|InParanoid|PANTHER|OrthoInspector',
 'AlgorithmsMatch': 9,
 'OutOfAlgorithms': 10,
 'IsBestScore': 'Yes',
 'IsBestRevScore': 'Yes'}

 
Example record:   
``` 
{'Gene1ID': 'WB:WBGene00011502',
  'Gene1Symbol': 'vps-53',
  'Gene1SpeciesTaxonID': 'NCBITaxon:6239',
  'Gene1SpeciesName': 'Caenorhabditis elegans',
  'Gene2ID': 'SGD:S000003566',
  'Gene2Symbol': 'VPS53',
  'Gene2SpeciesTaxonID': 'NCBITaxon:559292',
  'Gene2SpeciesName': 'Saccharomyces cerevisiae',
  'Algorithms': 'PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roundup|InParanoid|PANTHER|OrthoInspector',
  'AlgorithmsMatch': 9,
  'OutOfAlgorithms': 10,
  'IsBestScore': 'Yes',
  'IsBestRevScore': 'Yes'} 
  
```

In [65]:
data.columns.values

array(['Gene1ID', 'Gene1Symbol', 'Gene1SpeciesTaxonID',
       'Gene1SpeciesName', 'Gene2ID', 'Gene2Symbol',
       'Gene2SpeciesTaxonID', 'Gene2SpeciesName', 'Algorithms',
       'AlgorithmsMatch', 'OutOfAlgorithms', 'IsBestScore',
       'IsBestRevScore'], dtype=object)

In [66]:
data.head()

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
0,WB:WBGene00011502,vps-53,NCBITaxon:6239,Caenorhabditis elegans,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes
1,ZFIN:ZDB-GENE-041114-199,vps53,NCBITaxon:7955,Danio rerio,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roun...,8,10,Yes,Yes
2,RGD:1311391,Vps53,NCBITaxon:10116,Rattus norvegicus,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,OrthoFinder|Hieranoid|OMA|Ensembl Compara|InPa...,7,9,Yes,Yes
3,MGI:1915549,Vps53,NCBITaxon:10090,Mus musculus,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes
4,HGNC:25608,VPS53,NCBITaxon:9606,Homo sapiens,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes


---

## Parse Data
Now that we have loaded the data and correctly formatted it into records, we can loop through the new dictionary and access our data.

In [46]:
# Test Example Parse
#for rec in data2[:4]:
    #label_ids = rec.pop("Label IDs").split(";")
    #label_names = rec.pop("Label Names").split(";")
    #print(label_ids)
    #assert len(label_ids) == len(label_names)
    #labels = []
    #for i,_ in enumerate(label_ids):
        #labels.append({"id" : label_ids[i],
         #               "name" : label_names[i]})
   # _id = rec["Gene ID"]
    #rec = dict_convert(rec,keyfn=process_key)
    #doc = {"_id" : _id, "drug_labels" : labels}



In [71]:
# get keys from dataframe 

results={}
for rec in data_list[:4]:
    print(rec["Gene1SpeciesName"])
    #gene1_ids = rec.pop("Gene1ID").split(":")

#print(gene1_ids)


Caenorhabditis elegans
Danio rerio
Rattus norvegicus
Mus musculus


In [72]:
data.head()

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
0,WB:WBGene00011502,vps-53,NCBITaxon:6239,Caenorhabditis elegans,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes
1,ZFIN:ZDB-GENE-041114-199,vps53,NCBITaxon:7955,Danio rerio,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roun...,8,10,Yes,Yes
2,RGD:1311391,Vps53,NCBITaxon:10116,Rattus norvegicus,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,OrthoFinder|Hieranoid|OMA|Ensembl Compara|InPa...,7,9,Yes,Yes
3,MGI:1915549,Vps53,NCBITaxon:10090,Mus musculus,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes
4,HGNC:25608,VPS53,NCBITaxon:9606,Homo sapiens,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes


In [73]:
data.columns.values

array(['Gene1ID', 'Gene1Symbol', 'Gene1SpeciesTaxonID',
       'Gene1SpeciesName', 'Gene2ID', 'Gene2Symbol',
       'Gene2SpeciesTaxonID', 'Gene2SpeciesName', 'Algorithms',
       'AlgorithmsMatch', 'OutOfAlgorithms', 'IsBestScore',
       'IsBestRevScore'], dtype=object)

In [83]:
gene1_ids = data['Gene1ID']



In [85]:
len(data['Gene1Symbol'].unique().tolist())

81990

column: Gene1ID length: 100430
column: Gene1Symbol length: 81990
column: Gene1SpeciesTaxonID length: 7
['NCBITaxon:6239', 'NCBITaxon:7955', 'NCBITaxon:10116', 'NCBITaxon:10090', 'NCBITaxon:9606', 'NCBITaxon:7227', 'NCBITaxon:559292']
column: Gene1SpeciesName length: 7
['Caenorhabditis elegans', 'Danio rerio', 'Rattus norvegicus', 'Mus musculus', 'Homo sapiens', 'Drosophila melanogaster', 'Saccharomyces cerevisiae']
column: Gene2ID length: 100430
column: Gene2Symbol length: 81990
column: Gene2SpeciesTaxonID length: 7
['NCBITaxon:559292', 'NCBITaxon:7227', 'NCBITaxon:7955', 'NCBITaxon:10116', 'NCBITaxon:9606', 'NCBITaxon:6239', 'NCBITaxon:10090']
column: Gene2SpeciesName length: 7
['Saccharomyces cerevisiae', 'Drosophila melanogaster', 'Danio rerio', 'Rattus norvegicus', 'Homo sapiens', 'Caenorhabditis elegans', 'Mus musculus']
column: Algorithms length: 2529
column: AlgorithmsMatch length: 11
[9, 8, 7, 10, 6, 5, 3, 4, 2, 11, 1]
column: OutOfAlgorithms length: 3
[10, 9, 11]
column: IsBes

---